# 0 Module import

In [ ]:

import os
import numpy as np
import pandas as pd
import cv2
from tqdm.notebook import tqdm
from __future__ import print_function, division
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import albumentations
from albumentations.imgaug.transforms import IAAAffine#, ScaleX, ScaleY
import albumentations.pytorch
from albumentations.core.transforms_interface import ImageOnlyTransform

import pandas as pd
from sklearn.preprocessing import LabelEncoder
import cv2
cudnn.benchmark = True
from sklearn.metrics import f1_score
import pandas as pd
import torch
import cv2
from torch.nn import functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from transformers import AdamW, AutoModel, AutoTokenizer, ViTForImageClassification, ElectraModel, ElectraForSequenceClassification, ElectraTokenizer, BertForSequenceClassification, BertTokenizerFast, ViTConfig
from transformers import AutoModel,ViTModel,ViTFeatureExtractor
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import numpy as np
import glob
import gc
import pandas as pd
import numpy as np
import os
import random 
from transformers import AdamW
from torchvision import transforms
import timm
import albumentations
from albumentations.imgaug.transforms import IAAAffine

# 1. Data_Preparation

In [ ]:
#한글로 이미지 읽기 쓰기

def han_imread(ld_dir, color_mode = True) :
    img_array = np.fromfile(ld_dir, np.uint8)
    if color_mode == True :
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    else :
        img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
    return img

def han_imwrite(sv_dir, image) :
    new_img_name = sv_dir
    extension = os.path.splitext(new_img_name)[1] # 이미지 확장자
    result, encoded_img = cv2.imencode(extension, image)
    if result:
        with open(new_img_name, mode='w+b') as f:
            encoded_img.tofile(f)

root_dir = 'Data/train/' # 이미지 파일이 있는 디렉토리
os.makedirs('Data/Training_whole/PNG', exist_ok=True)
os.makedirs('Data/Training_whole/PNG_384', exist_ok=True) # 사이즈는 훈련할때 미리 원하는 사이즈로 있어야 빠름. 원하는 preprocessing이나 사이즈는 미리 넣어주세요
meta_dict_list = []
i = 0
for (root, dirs, files) in tqdm(os.walk(root_dir)): # 디렉토리 내의 모든 파일 순회
    for file_name in tqdm(files):
        if ('png' in file_name) or ('jpg' in file_name) or ('bmp' in file_name) : # 무슨 확장자가 있는지 몰라서 일단 썼습니다
            
            meta_dict = dict()
            png_img_name = os.path.basename(root) + '_' + file_name.split('.')[0] + '.png'
            meta_dict['id'] = i
            meta_dict['filename'] = png_img_name
            
            meta_dict['label'] = os.path.basename(root)
            meta_dict_list.append(meta_dict)
            
            img = han_imread(os.path.join(root, file_name))
            img_224 = cv2.resize(img, dsize = (384, 384), interpolation = cv2.INTER_AREA)

            han_imwrite( os.path.join('Data/Training_whole/PNG', png_img_name), img)
            han_imwrite( os.path.join('Data/Training_whole/PNG_384', png_img_name), img_224)
            i += 1

meta_train = pd.DataFrame(meta_dict_list)
meta_train.to_csv('train.csv', index = False)


# 2. Grid_search 함수 정의

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}
seed_everything(CFG['SEED']) # Seed 고정
def han_imread(ld_dir, color_mode = True) :
    img_array = np.fromfile(ld_dir, np.uint8)
    if color_mode == True :
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    else :
        img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
    return img

def han_imwrite(sv_dir, image) :
    new_img_name = sv_dir
    extension = os.path.splitext(new_img_name)[1] # 이미지 확장자
    result, encoded_img = cv2.imencode(extension, image)
    if result:
        with open(new_img_name, mode='w+b') as f:
            encoded_img.tofile(f)          

def make_aug(meta_train_i, std_len) :
    i_len = len(meta_train_i)
    meta_train_i_aug = meta_train_i.copy()
    num_concat = std_len//(i_len)-1
    num_rand = std_len-(num_concat+1)*(i_len)
    for n in range(num_concat) :
        meta_train_i_aug = pd.concat((meta_train_i_aug, meta_train_i))


    random_idx = random.sample(list(meta_train_i.index), num_rand)
    meta_train_i_aug = pd.concat((meta_train_i_aug, meta_train_i.loc[random_idx]))
    return meta_train_i_aug


def train_test_split_with_aug(meta_final, num_train_list, val_num_list, save_dir = None) :
    meta_final['task'] = 'Art'
    task_list = sorted(np.unique(meta_final['task']))
    # num_train = 1000
    # num_test = 200
    meta_test_list = []
    meta_train_list = []
    meta_val_list = []
    meta_rest_list = []

    for j, task in enumerate(task_list) :
        meta_j = meta_final[meta_final['task']==task]
        wall_list = sorted(np.unique(meta_j['class']))
    #     aug_conunt = aug_list[j]
        num_train = num_train_list[j]
        num_test = val_num_list[j]
        for i, wall in enumerate(wall_list) :

            meta_i = meta_j[meta_j['class']==wall]
            idx_list = random.sample(list(meta_i.index), len(meta_i))
            if len(meta_i) < num_test :
                num_test_aug = int(np.ceil(0.3 * len(meta_i)))
                meta_test_i = meta_i.loc[idx_list[:num_test_aug]]
                meta_test_i = make_aug(meta_test_i, num_test)

                meta_val_i = meta_test_i.iloc[:int(num_test/2),:]
                meta_test_i = meta_test_i.iloc[int(num_test/2):,:]
                meta_train_i = meta_i.loc[idx_list[num_test_aug:]]
                if len(meta_train_i) < num_train :
                    meta_train_i = make_aug(meta_train_i, num_train)
                    meta_train_i['aug'] = 'aug'
                    print(wall, 'aug!')
                else :
                    meta_train_i['aug'] = 'raw'

            else :
                meta_test_i = meta_i.loc[idx_list[:num_test]]
                meta_train_i = meta_i.loc[idx_list[num_test:]]
                meta_val_i = meta_test_i.iloc[:int(num_test/2),:]
                meta_test_i = meta_test_i.iloc[int(num_test/2):,:]
                if len(meta_train_i) < num_train :
                    print(wall, 'aug!')
                    meta_train_i = make_aug(meta_train_i, num_train)
                    meta_train_i['aug'] = 'aug'
                else:
                    meta_train_i['aug'] = 'raw'

            meta_val_list = meta_val_list + [meta_val_i]
            meta_test_list = meta_test_list + [meta_test_i]
            meta_train_list = meta_train_list + [meta_train_i]
    meta_val = pd.concat((meta_val_list))    
    meta_test = pd.concat((meta_test_list))
    meta_train = pd.concat((meta_train_list))
    if save_dir :
        os.makedirs(save_dir, exist_ok= True)
        meta_train.to_csv(os.path.join(save_dir, 'train.csv'))
        meta_test.to_csv(os.path.join(save_dir, 'test.csv'))
        meta_val.to_csv(os.path.join(save_dir, 'val.csv'))
    
    
    return meta_train, meta_val, meta_test
    # meta_rest = pd.concat((meta_rest_list))



class AugMix(ImageOnlyTransform):
    """Augmentations mix to Improve Robustness and Uncertainty.
    Args:
        image (np.ndarray): Raw input image of shape (h, w, c)
        severity (int): Severity of underlying augmentation operators.
        width (int): Width of augmentation chain
        depth (int): Depth of augmentation chain. -1 enables stochastic depth uniformly
          from [1, 3]
        alpha (float): Probability coefficient for Beta and Dirichlet distributions.
        augmentations (list of augmentations): Augmentations that need to mix and perform.
    Targets:
        image
    Image types:
        uint8, float32
    """

    def __init__(self, width=2, depth=2, alpha=0.5, augmentations=[], always_apply=False, p=0.5):
        super(AugMix, self).__init__(always_apply, p)
        self.width = width
        self.depth = depth
        self.alpha = alpha
        self.augmentations = augmentations
        self.ws = np.float32(np.random.dirichlet([self.alpha] * self.width))
        self.m = np.float32(np.random.beta(self.alpha, self.alpha))

    def apply_op(self, image, op):
        image = op(image=image)["image"]
        return image

    def apply(self, img, **params):
        mix = np.zeros_like(img)
        for i in range(self.width):
            image_aug = img.copy()

            for _ in range(self.depth):
                op = np.random.choice(self.augmentations)
                image_aug = self.apply_op(image_aug, op)

            mix = np.add(mix, self.ws[i] * image_aug, out=mix, casting="unsafe")

        mixed = (1 - self.m) * img + self.m * mix
        if img.dtype in ["uint8", "uint16", "uint32", "uint64"]:
            mixed = np.clip((mixed), 0, 255).astype(np.uint8)
        return mixed

    def get_transform_init_args_names(self):
        return ("width", "depth", "alpha")

def train_test_split_wo_aug(meta_final, split_ratio, save_dir = None) :
    meta_final['task'] = 'Art'
    task_list = sorted(np.unique(meta_final['task']))
    # num_train = 1000
    # num_test = 200
    meta_test_list = []
    meta_train_list = []
    meta_val_list = []
    meta_rest_list = []
    
    whole = summary(meta_final)
    Summary_table = whole.Summary
#     _, Summary_table, _  = make_summary(meta_final, show_fig = False) 
    Summary_table_test = (Summary_table*split_ratio).astype('int')
    Summary_table_train = Summary_table - Summary_table_test
    
    
    for j, task in enumerate(task_list) :
        meta_j = meta_final[meta_final['task']==task]
        wall_list = sorted(np.unique(meta_j['class']))
    #     aug_conunt = aug_list[j]

            
        for i, wall in enumerate(wall_list) :

            num_train = Summary_table_train.loc[task].loc[wall][0]
            num_test = Summary_table_test.loc[task].loc[wall][0]
            if num_test < 2 :
                num_test = 1

            meta_i = meta_j[meta_j['class']==wall]
            idx_list = random.sample(list(meta_i.index), len(meta_i))

            meta_test_i = meta_i.loc[idx_list[:num_test]]
            meta_train_i = meta_i.loc[idx_list[num_test:]]
            if num_test == 1 :
                meta_val_i = meta_test_i.copy()
            else : 
                meta_val_i = meta_test_i.iloc[:int(num_test/2),:]
                meta_test_i = meta_test_i.iloc[int(num_test/2):,:]

            meta_val_list = meta_val_list + [meta_val_i]
            meta_test_list = meta_test_list + [meta_test_i]
            meta_train_list = meta_train_list + [meta_train_i]
    meta_val = pd.concat((meta_val_list))    
    meta_test = pd.concat((meta_test_list))
    meta_train = pd.concat((meta_train_list))
    
    if save_dir :
        os.makedirs(save_dir, exist_ok= True)
        meta_train.to_csv(os.path.join(save_dir, 'train.csv'))
        meta_test.to_csv(os.path.join(save_dir, 'test.csv'))
        meta_val.to_csv(os.path.join(save_dir, 'val.csv'))
    
    return meta_train, meta_val, meta_test
    # meta_rest = pd.concat((meta_rest_list))

class summary() :
    def __init__(self, meta_final, show_fig = False) :
        self.meta_final = meta_final
        self.meta_final['task'] = 'Art'
        task_list = np.unique(self.meta_final['task'])
        task_df_dic = {}
        image_count_dic = {}
        count_final = 0
        Summary_list = []

        for task in task_list :
            task_df = self.meta_final[self.meta_final['task']==task]
            task_df_dic[task] = task_df
            haja_list = np.unique(task_df['class'])
            image_count = {}

            for haja in haja_list :
                Summary_dic = {}
                image_count[haja] = len(task_df[task_df['class']==haja])
                Summary_dic['task'] = task
                Summary_dic['class'] = haja
                Summary_dic['Count'] = len(task_df[task_df['class']==haja])
                Summary_list.append(Summary_dic)
            if show_fig == True :
                fig1, ax1 = plt.subplots()
                ax1.pie(image_count.values(),
                        labels = image_count.keys(),
                        shadow=True,
                        autopct = '%1.1f%%',
                        startangle=90)
                plt.title(task+', count : {}'.format(len(task_df)))
                plt.show()
            else :
                pass

            count = []
            for key, values in sorted(image_count.items()) :
                count.append(values)

            image_count_dic[task] = image_count
            count= np.array(count)
            count_final = count_final + count.sum()
            ratio = count / count.max()
            aug_list = 1/ratio
        self.Summary_list = pd.DataFrame(Summary_list)
        self.Summary = pd.DataFrame(Summary_list).groupby(['task','class']).sum()
        self.Summary_mean = pd.DataFrame(Summary_list).groupby(['task']).mean()
        self.Summary_min = pd.DataFrame(Summary_list).groupby(['task']).min().iloc[:,1:]
        self.Summary_max = pd.DataFrame(Summary_list).groupby(['task']).max().iloc[:,1:]
        return pd.DataFrame(Summary_list), Summary, Summary_avg, Summary_task

def export_img(img_root, row, train_mode = 'val', feature_extractor = None, input_size = 224) :
    
    img_path = os.path.join(img_root, str(os.path.basename(row[0])))    
    image_feature = han_imread(img_path)#cv2.imread(img_path)

    if train_mode == 'train' :
        transform = [albumentations.Resize(input_size,input_size,),
                            albumentations.HorizontalFlip(p=0.5), albumentations.VerticalFlip(p=0.5), 
                            albumentations.RandomBrightnessContrast(brightness_limit=(-0.3, 0.3), contrast_limit=0, p=0.3),
                            IAAAffine(scale = (0.9, 1.1), translate_percent = (0,0), rotate = (0,20), shear = 0, cval = 0, p=1.0,),
#                             OneOf([albumentations.ShiftScaleRotate(always_apply=True), albumentations.GaussNoise(always_apply=True)]),
                            albumentations.Cutout(always_apply=True),]
                            #ScaleX(0.9, 1.1), ScaleY(0.9, 1.1),]) # mode = 'reflect')])#,albumentations.RandomCrop(crop_x, crop_y)]) 
        transform = albumentations.Compose([AugMix(width=3, depth=2, alpha=.2, p=1., augmentations=transform),])
        
        augmented = transform(image=image_feature) 
        image_feature = augmented['image']
    elif train_mode == 'val' :
        transform = albumentations.Compose([albumentations.Resize(input_size,input_size)])    
        augmented = transform(image=image_feature) 
        image_feature = augmented['image']
    elif train_mode == 'test' :
        transform = albumentations.Compose([albumentations.Resize(input_size,input_size)])    
        augmented = transform(image=image_feature) 
        image_feature = augmented['image']

    if feature_extractor :
        image_feature = feature_extractor(images=image_feature, return_tensors="pt")
        image_feature = image_feature['pixel_values'][0]
    else :
#         image_feature = cv2.resize(image_feature, (input_size, input_size), interpolation = cv2.INTER_AREA)
        image_feature = image_feature/255
        image_feature = np.transpose(image_feature,(2,0,1)).astype(np.float32)
    return image_feature


def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
class img_Dataset(Dataset):

    def __init__(self, df, img_root, encoder, train_mode = 'val', feature_extractor = None, input_size = 224):
        # 일부 값중에 NaN이 있음...
        self.dataset = df.dropna(axis=0)
        self.img_root = img_root
        self.train_mode = train_mode
        self.feature_extractor = feature_extractor
        self.input_size = input_size
        # 중복제거
        #self.dataset.drop_duplicates(subset=['document'], inplace=True)
        self.dataset['label'] =  encoder.transform(self.dataset['class'])
        self.dataset = self.dataset[['filename', 'label']]
        #     self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")
        print('found {} data'.format(len(self.dataset)))
        print(self.dataset.describe())

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset.iloc[idx].values
        img_path = row[0]
        y = row[1]
        image_feature = export_img(self.img_root, row, self.train_mode, self.feature_extractor, self.input_size) 
        return {
            'image_feature': image_feature,
            'y':y
        }
    
    
def infer(model, test_loader, device) :
    model.eval()
    correct = 0
    batches = 0
    total = 0
    total_loss = 0.0
    with torch.no_grad() :
        for i, batch in enumerate(tqdm(test_loader)):
            img_feature = batch['image_feature'].to(device)
            y_batch = batch["y"].to(device, dtype=torch.int64)
            y_pred = model(img_feature)
            _, predicted = torch.max(y_pred, 1)
            act_vec = torch.nn.Softmax(1)(y_pred)
            batches += 1
            if i == 0 :
                y_batch_whole = y_batch.cpu()
                y_pred_whole = predicted.cpu()
                act_vec_whole = act_vec.cpu()
            else :
                y_batch_whole = torch.cat((y_batch_whole, y_batch.cpu()))
                y_pred_whole = torch.cat((y_pred_whole, predicted.cpu()))
                act_vec_whole = torch.cat((act_vec_whole, act_vec.cpu()))
    return y_batch_whole, y_pred_whole, act_vec_whole

def train_test_split_k(df, k) :
    df_train = df[df['kfold']!=k]
    df_val = df[df['kfold']==k]
    return df_train, df_val

    
def load_dataset(path, k, random_state, reset_k = False) :
    df = pd.read_csv(path)
    encoder = LabelEncoder()
    encoder.fit(sorted((pd.read_csv(path))['class'].unique()))
    if reset_k == True :
        folds = StratifiedKFold(n_splits=k, random_state=random_state, shuffle=True)
        df['kfold'] = -1
        for i in range(k):
            df_idx, valid_idx = list(folds.split(df.values, df['class']))[i]
            valid = df.iloc[valid_idx]
            df.loc[df[df.index.isin(valid.index) == True].index.to_list(), 'kfold'] = i
        df.to_csv('Data/train_k.csv')
    else :
        pass
    return df, encoder
                
# https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html 참고
# https://github.com/rwightman/pytorch-image-models/blob/main/results/results-imagenet-a-clean.csv timm 모델 참고
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True, device = 'cuda'):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0
    
    if model_name in ['cait_m36_384' ] :
        model_ft = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
        input_size = 384
        return model_ft.to(device), input_size, None


class fine_tuning : 
    def __init__(self, img_root, exp, file, condition, cv_k, model, batch_size,  lr, decay, aug_num, save_interval,
                 train_path, batch_size_test = 16, max_epoch = 20, early_stopping =None) : 
        
        self.exp = exp
        self.file = file
        self.condition = condition
        self.cv_k = cv_k
        self.file_path = os.path.join(self.file, self.condition)
        self.file_k_path = os.path.join(self.file_path, str(self.cv_k))
        os.makedirs(self.file_k_path, exist_ok=True)
        self.max_epoch = max_epoch
        self.b = batch_size
        self.d = decay
        self.aug_num = aug_num
        self.lr = lr
        self.save_interval = save_interval
        
        df = pd.read_csv(train_path)
        self.encoder = LabelEncoder()
        self.encoder.fit(sorted((df)['class'].unique()))
        
        self.df_train, self.df_val = train_test_split_k(df, cv_k) # 0~3 까지의 값
        
        if self.aug_num > 1 : 
            self.df_train['task'] = 'Art'
            self.df_train, _, _ = train_test_split_with_aug(self.df_train, [aug_num], [0], save_dir = None)
            self.df_train = self.df_train.drop(['task', 'aug'], axis = 1)
        
        if torch.cuda.is_available() == True :
            self.device = torch.device("cuda")
        else :
            self.device = torch.device("cpu")
            
            
        self.model, self.input_size, self.extractor = initialize_model(model,len(self.encoder.classes_), feature_extract = False , use_pretrained=True, device = self.device)
        train_dataset = img_Dataset(self.df_train, img_root, self.encoder, 'train', self.extractor, self.input_size)
        val_dataset = img_Dataset(self.df_val, img_root, self.encoder, 'val', self.extractor, self.input_size)
        
#         self.optimizer = optimizer = AdamW(self.model.parameters(), lr=self.lr, weight_decay=self.d, correct_bias=True) # AdamW에서 bias correction 과정만 생략해주시면 BERTAdam이 됩니다!
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.d)
        #self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer , mode='max', factor=0.1, patience=2, threshold=1e-7)
        self.train_loader = DataLoader(train_dataset, batch_size=self.b, shuffle=True, num_workers = 4)
        self.val_loader = DataLoader(val_dataset, batch_size=batch_size_test, shuffle=False, num_workers = 4)

        
    def train(self) :
        self.history_list = []
        best_f1_val = 0
        
        for i in range(self.max_epoch):
            print("Epoch {}".format(i+1))
            history = dict()
            loss_train, acc_train, f1_train = self.train_one_epoch(self.train_loader, True)
            print("train_Loss:{:.3f}".format(loss_train), "train_Accuracy:{:.3f}".format(acc_train.item()), "train_F1:{:.3f}".format(f1_train))
            with torch.no_grad() :
                loss_val, acc_val, f1_val = self.train_one_epoch(self.val_loader, False)
            print("val_Loss:{:.3f}".format(loss_val), "val_Accuracy:{:.3f}".format(acc_val.item()), "val_F1:{:.3f}".format(f1_val))
            history['loss'] =  loss_train
            history['accuracy'] =  acc_train.item()
            history['f1'] =  f1_train
            history['val_loss'] = loss_val
            history['val_accuracy'] = acc_val.item()
            history['val_f1'] =  f1_val
            #self.scheduler.step(f1_val)
            self.history_list.append(history)
            if i % self.save_interval == 0:
                if f1_val > best_f1_val :
                    print('val_f1 improved!! saving model...')
                    targetPattern = r"{}/*.pt".format(self.file_k_path)
                    h5_list = glob.glob(targetPattern)
                    if len(h5_list) >= 1 :
                        print('previous model deleted')
                        os.remove(h5_list[0])
                    checkpoint_path = os.path.join(self.file_k_path,
                                                   self.condition + '_{:02d}-{:.3f}-{:.3f}-{:.3f}.pt'.format(i, loss_val, f1_train, f1_val))#acc_val.item()))
                    torch.save(self.model.state_dict(), checkpoint_path)
                    best_f1_val = f1_val#acc_val.item()

                else :
                    print('val_accuracy is not improved...')
        self.history = pd.DataFrame(self.history_list)

    def train_one_epoch(self, train_loader, train_mode) :
        if train_mode == True :
            self.model.train()
        else :
            self.model.eval()
        
        correct = 0
        batches = 0
        total = 0
        total_loss = 0.0
        true_labels = []
        model_preds = []
        for batch in tqdm(train_loader):
            self.optimizer.zero_grad()
            img_feature = batch['image_feature'].to(self.device)
            y_batch = batch["y"].to(self.device, dtype=torch.int64)
            y_pred = self.model(img_feature)
            true_labels += y_batch.detach().cpu().numpy().tolist()
            model_preds += y_pred.argmax(1).detach().cpu().numpy().tolist()
            loss = F.cross_entropy(y_pred, y_batch)
            if train_mode == True :
                loss.backward()
                self.optimizer.step()
                
            total_loss += loss.item() * len(y_batch)
            _, predicted = torch.max(y_pred, 1)
            correct += (predicted == y_batch).sum()
            total += len(y_batch)
            batches += 1
            
            if (batches % 100 == 0) and (train_mode == True) :
                print("Loss:", loss.item(), "Accuracy:", correct.float() / total)
        total_loss = total_loss / len(train_loader.dataset)
        accuracy = correct.float() / len(train_loader.dataset)
        f1 = f1_score(true_labels, model_preds, average="macro")
        return total_loss, accuracy, f1

    def save_history(self) :
        self.show_history()
        self.save_summary()
        
    def show_history(self) :
        fig = plt.figure(figsize=(18,6))
        ax1 = fig.add_subplot(1, 3, 1)
        plt.plot(self.history['loss'])
        plt.plot(self.history['val_loss'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        ax2 = fig.add_subplot(1, 3, 2)
        plt.plot(self.history['accuracy'])
        plt.plot(self.history['val_accuracy'])
        plt.xlabel('Epoch')
        ax3 = fig.add_subplot(1, 3, 3)
        plt.plot(self.history['f1'])
        plt.plot(self.history['val_f1'])
        plt.xlabel('Epoch')
        plt.ylabel('f1')
        plt.savefig(os.path.join(self.file_k_path, 'Loss-acc-f1_plot.png'))
        self.history.to_csv(os.path.join(self.file_k_path,'history.csv'))
        
        
    def save_summary(self) :
        A = np.array(self.history.index+1)
        index = list(A[A%self.save_interval==0]-1)
        Result = self.history[['loss','val_loss','accuracy','val_accuracy', 'f1', 'val_f1']].iloc[index ,:].reset_index()
        Result['index'] = Result['index']+1
        Result.rename(columns = {'index' : 'epoch'}, inplace = True)
        Result['index']=self.exp
        Result['cv_k']=self.cv_k
        Result['batch_size']=self.b
        Result['learning_rate']=self.lr
        Result['L2_decay']=self.d
        Result['aug_num']=self.aug_num
        Result = Result[['index','cv_k', 'batch_size', 'learning_rate','L2_decay', 'aug_num', 'epoch','loss', 
                         'val_loss','accuracy','val_accuracy', 'f1','val_f1']]
        if not ('Result.csv' in os.listdir(self.file)):
            Result.to_csv(os.path.join(self.file,'Result.csv'))
        else:
            Result_0 = pd.read_csv(os.path.join(self.file,'Result.csv'), index_col=0)
            Result = pd.concat([Result_0, Result],axis=0)
            Result.to_csv(os.path.join(self.file,'Result.csv'))
                    
    def find_best_epoch(self) : 
        targetPattern = r"{}/*.pt".format(self.file_k_path)
        h5_list = glob.glob(targetPattern)
        f1_list = []
        for h5 in h5_list:
            f1_list.append(float(h5.split('-')[-1][:5])) 
        self.best_model_path = h5_list[(np.argmax(np.array(f1_list)))]
        self.max_epoch = int((self.best_model_path.split('-')[-4].split('_'))[-1])
        return np.max(np.array(f1_list))        


class inference : 
    def __init__(self, model, encoder, file_path, cv_k, test_path, val_dataloader, model_path, max_epoch,img_root, extractor, input_size) :
        self.encoder = encoder
        self.cv_k = cv_k
        self.dataframe_test = pd.read_csv(test_path)#.iloc[:,1:]
        self.file_path = file_path
        self.file_k_path = os.path.join(self.file_path, str(cv_k))
        self.model_path = model_path
        self.max_epoch  = max_epoch
        self.test_df = pd.read_csv(test_path)
        
        def export_filename(x) :
            return os.path.basename(x)
        
        self.test_df['filename'] = self.test_df['img_path'].apply(export_filename)
        self.test_df['class'] = '가구수정'
        self.sub_df = pd.read_csv('Data/sample_submission.csv')

        if torch.cuda.is_available() == True :
            self.device = torch.device("cuda")
        else :
            self.device = torch.device("cpu")
       
        self.model = model
        test_dataset = img_Dataset(self.test_df, img_root, encoder, 'test', extractor, input_size)
        self.model.load_state_dict(torch.load(self.model_path)) 
        self.test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers = 4)
        self.predict()

    def predict(self) :
        _, pred_test, self.act_vec_test = infer(self.model, self.test_loader, self.device) 
        pred_list = self.encoder.inverse_transform(pred_test.cpu().numpy())
        self.test_df['class'] = pred_list 
        self.sub_df['label'] = pred_list 
        self.test_df.to_csv(os.path.join(self.file_path, 'test_cv_{}.csv'.format(self.cv_k)), index = False)
        self.sub_df.to_csv(os.path.join(self.file_path, 'sub_cv_{}.csv'.format(self.cv_k)), index = False)                         

    def pred_val(self, val_df, val_dataloader) :
        _, pred_val, _ = infer(self.model, val_dataloader, self.device) 
        pred_val_list = self.encoder.inverse_transform(pred_val.cpu().numpy())
        val_df['class'] = pred_val_list
        return val_df

class grid_search :
    def __init__(self, train_path, test_path, img_root, img_test_root, file, cv_k_list, md_list, batch_size_list, lr_list, 
                 decay_list, aug_num_list, max_epoch, interval, train, initial_exp, start_exp) :
        self.img_root = img_root
        self.img_test_root = img_test_root
        self.train_path = train_path  
        self.test_path =test_path 
        self.file = file
        self.cv_k_list = cv_k_list
        self.md_list = md_list
        self.batch_size_list = batch_size_list
        self.lr_list = lr_list
        self.decay_list = decay_list
        self.aug_num_list = aug_num_list
        self.max_epoch = max_epoch
        self.save_interval = interval
        self.train = train
        self.exp = initial_exp
        self.start_exp = start_exp
        self.run()

    def run(self) :
        for md in self.md_list :
            for b in self.batch_size_list :
                for lr in self.lr_list : 
                    for d in self.decay_list :
                        for aug_num in self.aug_num_list :

                            self.Result = dict()
                            self.Result_list = []
                            self.Result['index']=self.exp
                            self.Result['model']=md
                            self.Result['batch_size']=b
                            self.Result['learning_rate']=lr
                            self.Result['L2_decay']=d
                            self.Result['aug_num']=aug_num

                            condition = '{}_md({})b({})L({})d({})an({})'.format(self.exp, md, b, lr, d, aug_num)
                            file_path = os.path.join(self.file, condition)
                            if self.exp < self.start_exp :
                                print('{} passed'.format(condition))
                                self.exp = self.exp + 1
                                pass
                            else :

                                act_vec_list = []
                                val_df_wrong_list = []
                                for cv_k in self.cv_k_list : 

    #                                                     try :
                                    print(condition)
                                    self.my_bert = fine_tuning (self.img_root, self.exp, self.file, condition, cv_k,
                                                                md, b, lr, d, aug_num, self.save_interval,
                                                                self.train_path, batch_size_test = 16,
                                                                max_epoch = self.max_epoch, early_stopping =None)

                                    if self.train == True :
                                        if 'history.csv' in os.listdir(self.my_bert.file_k_path) :
                                            print('Already trained, pass!!!')
                                        else:
                                            self.my_bert.train()
                                            self.my_bert.save_history()
                                    else :
                                        pass
                                    max_f1 = self.my_bert.find_best_epoch()
                                    self.Result['f1_cv_{}'.format(cv_k)] = max_f1

                                    self.my_bert_inf = inference(self.my_bert.model, self.my_bert.encoder, file_path, cv_k,
                                                                 self.test_path, self.my_bert.val_loader,
                                                                 self.my_bert.best_model_path,
                                                                 self.my_bert.max_epoch, self.img_test_root, self.my_bert.extractor, self.my_bert.input_size)

                                    val_df_wrong = self.my_bert_inf.pred_val(self.my_bert.df_val, self.my_bert.val_loader)
                                    val_df_wrong_list.append(val_df_wrong)


                                    if cv_k == 0 :
                                        act_vec_enss = self.my_bert_inf.act_vec_test.numpy().copy()
                                    else :
                                        act_vec_enss = act_vec_enss + self.my_bert_inf.act_vec_test.numpy()

                                val_df_wrong = pd.concat(val_df_wrong_list)
                                val_df_wrong.sort_values('id')
                                val_df_wrong.to_excel(os.path.join(self.my_bert.file_path, 'train_df_wrong.xlsx'), index = False)

                                act_vec_enss = act_vec_enss / 4
                                pred_enss = np.argmax(act_vec_enss, 1)
                                pred_list = self.my_bert.encoder.inverse_transform(pred_enss)
                                self.my_bert_inf.sub_df['label'] = pred_list
                                self.my_bert_inf.sub_df.to_csv(os.path.join(self.my_bert.file_path, 'sub_cv_enss.csv'), index = False)  
                                self.Summary = pd.DataFrame([self.Result])
                                if 'Summary_cv.xlsx' in os.listdir(self.file) :
                                    Summary_0 = pd.read_excel(os.path.join(self.file, 'Summary_cv.xlsx'), engine='openpyxl')
                                    self.Summary = pd.concat((Summary_0, self.Summary))
                                    self.Summary = self.Summary.drop_duplicates('index', keep='last')
                                    self.Summary.to_excel(os.path.join(self.file, 'Summary_cv.xlsx'), index = False)
                                else : 
                                    self.Summary.to_excel(os.path.join(self.file, 'Summary_cv.xlsx'), index = False)



                                self.exp = self.exp + 1

                                gc.collect()
                                torch.cuda.reset_max_memory_allocated(device=torch.device("cuda"))
                                torch.cuda.empty_cache()


# 3. Run

In [ ]:
import os
# from train_torch_augmix import *
if __name__ == '__main__' :
    
    train_path = r"Data/train_k.csv"
    test_path = r"Data/test.csv"
    img_root = r"Data/Training_whole/PNG_384"
    img_test_root = r"Data/test"
    file = 'training/exp_cait_m36_384_ft_augmix'
    cv_k_list = [0, 1, 2, 3]
    model_list = ['cait_m36_384']

    lr_list = [1e-6]#, 
    decay_list = [0]#,
    batch_size_list = [1]
    aug_num_list = [900]
    max_epoch = 6
    interval = 1
    train = True
    initial_exp = 0
    start_exp = 0

    my_gs = grid_search(train_path, test_path, img_root, img_test_root, file, cv_k_list, model_list, batch_size_list, lr_list,
                        decay_list, aug_num_list, max_epoch, interval, train, initial_exp, start_exp)
